In [1]:
import numpy as np
import pandas as pd
from yaml import safe_load
import os 
from tqdm import tqdm



C:\Users\user\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
filenames =[]
for file in os.listdir('data'):
    filenames.append(os.path.join('data',file))

In [3]:
filenames[0:5]

['data\\1001349.yaml',
 'data\\1001351.yaml',
 'data\\1001353.yaml',
 'data\\1004729.yaml',
 'data\\1007655.yaml']

In [4]:
import pandas as pd 

In [5]:
print(pd.__version__)

2.2.2


In [6]:
pip install --upgrade pandas

In [7]:
import numpy as np
import pandas as pd
from yaml import safe_load
import os 
from tqdm import tqdm

In [8]:
filenames =[]
for file in os.listdir('data'):
    filenames.append(os.path.join('data',file))

In [9]:
filenames[0:5]

['data\\1001349.yaml',
 'data\\1001351.yaml',
 'data\\1001353.yaml',
 'data\\1004729.yaml',
 'data\\1007655.yaml']

In [ ]:
import pandas as pd
final_df =pd.DataFrame()
counter =1
for file in tqdm(filenames):
    with open(file,'r')as f:
        df = pd.json_normalize(safe_load(f))
        df['match_id']=counter
        final_df = final_df._append(df)
        counter+=1
        
final_df        

 11%|████████▍                                                                      | 151/1419 [00:38<04:35,  4.60it/s]

In [ ]:
backup = final_df.copy()

In [ ]:
final_df

In [ ]:
final_df.drop(columns=[
    'meta.data_version',
    'meta.created',
    'meta.revision',
    'info.outcome.bowl_out',
    'info.bowl_out',
    'info.supersubs.South Africa',
    'info.supersubs.New Zealand',
    'info.outcome.eliminator',
    'info.outcome.result',
    'info.outcome.method',
    'info.neutral_venue',
    'info.match_type_number',
    'info.outcome.by.runs',
    'info.outcome.by.wickets'
],inplace=True)

In [ ]:
final_df

In [ ]:
final_df['info.gender'].value_counts()

In [ ]:
final_df=final_df[final_df['info.gender']=='male']
final_df.drop(columns=['info.gender'],inplace=True)
final_df

In [ ]:
final_df['info.match_type'].value_counts()

In [ ]:
final_df['info.overs'].value_counts()

In [ ]:
final_df=final_df[final_df['info.overs']==20]
final_df.drop(columns=['info.overs','info.match_type'],inplace=True)
final_df

In [ ]:
import pickle
pickle.dump(final_df,open('dataset_level1.pkl','wb'))

In [ ]:
matches=pickle.load(open('dataset_level1.pkl','rb'))
matches

In [ ]:
matches.iloc[0]['innings']

In [ ]:
matches.iloc[0]['innings'][0]['1st innings']['deliveries']

In [ ]:
import pandas as pd
count =1
delivery_df=pd.DataFrame()
for index,row in matches.iterrows():
    if count in[75,108,150,180,268,360,443,458,584,748,982,1052,1111,1226,1345]:
        count+=1
        continue
    count+=1
    ball_of_match=[]
    batsman=[]
    bowler=[]
    runs=[]
    player_of_dismissed=[]
    teams=[]
    batting_team=[]
    match_id=[]
    city=[]
    venue=[]
    for ball in row['innings'][0]['1st innings']['deliveries']:
        for key in ball.keys():
            match_id.append(count)
            batting_team.append(row['innings'][0]['1st innings']['team'])
            teams.append(row['info.teams'])
            ball_of_match.append(key)
            batsman.append(ball[key]['batsman'])
            bowler.append(ball[key]['bowler'])
            runs.append(ball[key]['runs']['total'])
            city.append(row['info.city'])
            venue.append(row['info.venue'])
            try:
                player_of_dismissed.append(ball[key]['wicket']['player_out'])
            except:
                player_of_dismissed.append('0')
    loop_df  =pd.DataFrame({
        'match_id':match_id,
        'teams':teams,
        'batting_team':batting_team,
        'ball':ball_of_match,
        'batsman':batsman,
        'bowler':bowler,
        'runs':runs,
        'player_dismissed':player_of_dismissed,
        'city':city,
        'venue':venue
    }) 
    delivery_df =delivery_df._append(loop_df)
                
            

In [ ]:
delivery_df

In [ ]:
def bowl(row):
    for team in row['teams']:
        if team !=row['batting_team']:
            return team

In [ ]:
delivery_df['bowling_team']=delivery_df.apply(bowl,axis=1)


In [ ]:
delivery_df

In [ ]:
delivery_df.drop(columns=['teams'],inplace=True)

In [ ]:
delivery_df['batting_team'].unique()

In [ ]:
delivery_df

In [ ]:
teams =[
    'Australia'
    'India'
    'Bangladesh'
    'New Zealand'
    'South Africa'
    'England'
    'West Indies'
    'Afganistan'
    'Pakistan'
    'Sri Lanka'
]

In [ ]:
delivery_df = delivery_df[delivery_df['batting_team'].isin(teams)]
delivery_df = delivery_df[delivery_df['bowling_team'].isin(teams)]

In [ ]:
delivery_df

In [ ]:
output = delivery_df[['match_id','batting_team','bowling_team','ball','runs','player_dismissed','city','venue']]

In [ ]:
output